In [1]:
%run data.py

In [2]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = load_imdb(final=False)

# Variable batch size

In [17]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

## Training data

In [39]:
# set a maximum number of tokens
k = 0
for i in range(20000):
    k += len(x_train[i])
token_perbatch = k/500
# generate batches
i = 0
batch = []
target = []
batch_number = 0
while i < 20000: # loop within training data
    token_number = 0
    tensor_x = []
    tensor_y = []
# Generate batch within a maximum number of tokens & loop within training data
    while (token_number < token_perbatch) & (i <20000): 
        tensor_x.append(torch.tensor(x_train[i], dtype=torch.long))
        token_number += len(x_train[i])
        tensor_y.append(y_train[i])
        i += 1
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    batch.append(tensor_pad)
    target.append(torch.tensor(tensor_y, dtype=torch.long))
    batch_number += 1

## validation set

In [47]:
# set a maximum number of tokens
k = 0
for i in range(5000):
    k += len(x_val[i])
token_perbatch = k/500
# generate batches
i = 0
v_batch = []
v_target = []
v_batch_number = 0
while i < 5000: # loop within training data
    token_number = 0
    tensor_x = []
    tensor_y = []
# Generate batch within a maximum number of tokens & loop within training data
    while (token_number < token_perbatch) & (i <5000): 
        tensor_x.append(torch.tensor(x_val[i], dtype=torch.long))
        token_number += len(x_val[i])
        tensor_y.append(y_val[i])
        i += 1
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    v_batch.append(tensor_pad)
    v_target.append(torch.tensor(tensor_y, dtype=torch.long))
    v_batch_number += 1

## Test 

In [41]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(300,300)  
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.max(F.relu(self.fc1(x)),1)
        x = self.fc2(x.values)
        return x

net = Net()
print(net)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [42]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(),betas=(0.9, 0.999), lr=0.0005)

In [43]:
for epoch in range(1): 
    for j in range(batch_number):
        outputs = net(batch[j])
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j])
        loss.backward()
        optimizer.step()
        print(loss.item())

print('Finished Training')

0.7028366923332214
0.6721299290657043
0.6936236023902893
0.6816436648368835
0.6804319620132446
0.6437385082244873
0.6807283759117126
0.6724631786346436
0.6500222682952881
0.6572632789611816
0.6541025042533875
0.6454006433486938
0.6652977466583252
0.6442278623580933
0.6335259079933167
0.6338663101196289
0.6492671370506287
0.6158459186553955
0.6196764707565308
0.6595302820205688
0.6278300285339355
0.6226454377174377
0.6307505369186401
0.6310650706291199
0.5991341471672058
0.5844296813011169
0.6189343333244324
0.6082825064659119
0.5960353016853333
0.6332985758781433
0.581642746925354
0.578914999961853
0.596298336982727
0.5860658884048462
0.5939655303955078
0.5807291269302368
0.5879064798355103
0.5610102415084839
0.6360515356063843
0.517217218875885
0.6883792877197266
0.4967830777168274
0.5647851824760437
0.5592250227928162
0.5597575902938843
0.5338048934936523
0.614253580570221
0.4949875473976135
0.6175767779350281
0.5285652875900269
0.5175122022628784
0.5595317482948303
0.588388860225677

0.39734116196632385
0.5404792428016663
0.5739432573318481
0.4417877197265625
0.4462341368198395
0.39978542923927307
0.35246890783309937
0.5008898973464966
0.4136865735054016
0.4144027531147003
0.5057533979415894
0.5483372807502747
0.5034946799278259
0.5516241192817688
0.6191729307174683
0.3816699683666229
0.4795386493206024
0.5313207507133484
0.3596089780330658
0.5396372675895691
0.40056514739990234
0.49908336997032166
0.5205144286155701
0.4412287175655365
0.40901848673820496
0.4859267771244049
0.5963524580001831
0.4916599988937378
0.4983563721179962
0.4474765360355377
0.37000465393066406
0.5182873010635376
0.3856698274612427
0.4686160087585449
0.4523746967315674
0.39500805735588074
0.5110557675361633
0.39143964648246765
0.5321423411369324
0.40006494522094727
0.35597267746925354
0.4355586767196655
0.5129756331443787
0.41239988803863525
0.4611127972602844
0.4041505753993988
0.5435357093811035
0.6165508031845093
0.5161910057067871
0.5539754629135132
0.6547929644584656
0.47829562425613403

In [48]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(v_batch_number):
        outputs = net(v_batch[j])
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j]).sum().item()

print('Accuracy of the network on the 5000 test: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 5000 test: 84 %
